In [4]:
import numpy as np
import cv2 as cv
import os
from copy import deepcopy

In [5]:
def show_image(title, image):
    image = cv.resize(image, (0,0), fx=0.2, fy=0.2)
    cv.imshow(title, image)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [6]:
def find_color_values_using_trackbar(frame):
    def nothing(x):
        pass

    cv.namedWindow('Trackbars')
    cv.createTrackbar('L - H', 'Trackbars', 0, 179, nothing)
    cv.createTrackbar('L - S', 'Trackbars', 0, 255, nothing)
    cv.createTrackbar('L - V', 'Trackbars', 0, 255, nothing)
    cv.createTrackbar('U - H', 'Trackbars', 179, 179, nothing)
    cv.createTrackbar('U - S', 'Trackbars', 255, 255, nothing)
    cv.createTrackbar('U - V', 'Trackbars', 255, 255, nothing)

    while True:
        l_h = cv.getTrackbarPos('L - H', 'Trackbars')
        l_s = cv.getTrackbarPos('L - S', 'Trackbars')
        l_v = cv.getTrackbarPos('L - V', 'Trackbars')
        u_h = cv.getTrackbarPos('U - H', 'Trackbars')
        u_s = cv.getTrackbarPos('U - S', 'Trackbars')
        u_v = cv.getTrackbarPos('U - V', 'Trackbars')

        l = np.array([l_h, l_s, l_v])
        u = np.array([u_h, u_s, u_v])

        frame_hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
        mask_table_hsv = cv.inRange(frame_hsv, l, u)

        res = cv.bitwise_and(frame, frame, mask=mask_table_hsv)

        # Resize the frame, mask, and result for display
        scale_percent = 20  # percent of original size
        width = int(frame.shape[1] * scale_percent / 100)
        height = int(frame.shape[0] * scale_percent / 100)
        dim = (width, height)

        resized_frame = cv.resize(frame, dim, interpolation=cv.INTER_AREA)
        resized_mask = cv.resize(mask_table_hsv, dim, interpolation=cv.INTER_AREA)
        resized_res = cv.resize(res, dim, interpolation=cv.INTER_AREA)

        cv.imshow("Frame", resized_frame)
        cv.imshow("Mask", resized_mask)
        cv.imshow("Result", resized_res)

        if cv.waitKey(1) & 0xFF == 27:
            break

    cv.destroyAllWindows()

In [4]:
# img = cv.imread("antrenare/1_01.jpg")
# find_color_values_using_trackbar(img)

In [7]:
width = 2800
height = 2800
square_size = 200

In [8]:
def get_mask_table_hsv(img):
    # pt marginea tablei
	l = np.array([18, 128, 123])
	u = np.array([121, 255, 255])

	img_hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)
	mask_hsv = cv.inRange(img_hsv, l, u)

	# show_image('img_initial', img)
	# show_image('mask_hsv', mask_hsv)
	return mask_hsv

In [9]:
def get_full_board(img):
	mask_hsv = get_mask_table_hsv(img)
	img_copy = img.copy()
	contours, _ = cv.findContours(mask_hsv, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
	# cv.drawContours(img_copy, contours, -1, (0, 255, 0), 10)
	for c in contours:
		area = cv.contourArea(c)
		x, y, w, h = cv.boundingRect(c)
		cv.rectangle(img_copy, (x, y), (x+w, y+h), (0, 255, 0), 10)

	# show_image('img', img_copy)

	max_area = 0

	for i in range(len(contours)):
		if len(contours[i]) > 3:
			possible_top_left = None
			possible_bottom_right = None
			for point in contours[i].squeeze():
				if possible_top_left is None or point[0] + point[1] < possible_top_left[0] + possible_top_left[1]:
					possible_top_left = point

				if possible_bottom_right is None or point[0] + point[1] > possible_bottom_right[0] + possible_bottom_right[1]:
					possible_bottom_right = point

			diff = np.diff(contours[i].squeeze(), axis=1)	# perechi de y - x
			possible_top_right = contours[i].squeeze()[np.argmin(diff)]
			possible_bottom_left = contours[i].squeeze()[np.argmax(diff)]
			if cv.contourArea(np.array([[possible_top_left], [possible_top_right], [possible_bottom_right], [possible_bottom_left]])) > max_area:
				max_area = cv.contourArea(np.array([[possible_top_left], [possible_top_right], [possible_bottom_right], [possible_bottom_left]]))
				top_left = possible_top_left
				bottom_right = possible_bottom_right
				top_right = possible_top_right
				bottom_left = possible_bottom_left

	cv.circle(img_copy, tuple(top_left), 20, (0, 0, 255), -1)
	cv.circle(img_copy, tuple(top_right), 20, (0, 0, 255), -1)
	cv.circle(img_copy, tuple(bottom_left), 20, (0, 0, 255), -1)
	cv.circle(img_copy, tuple(bottom_right), 20, (0, 0, 255), -1)
	# show_image("detected corners", img_copy)

	puzzle = np.array((top_left, top_right, bottom_right, bottom_left), dtype='float32')
	destination = np.array(([0, 0], [width, 0], [width, height], [0, height]), dtype='float32')
	M = cv.getPerspectiveTransform(puzzle, destination)
	result = cv.warpPerspective(img, M, (width, height), flags=cv.INTER_LINEAR)
	# show_image("result", result)
	return result

In [10]:
empty_board = [
    ['x3', 'e', 'e', 'e', 'e', 'e', 'x3', 'x3', 'e', 'e', 'e', 'e', 'e', 'x3'],
    ['e', 'x2', 'e', 'e', '/', 'e', 'e', 'e', 'e', '/', 'e', 'e', 'x2', 'e'],
    ['e', 'e', 'x2', 'e', 'e', '-', 'e', 'e', '-', 'e', 'e', 'x2', 'e', 'e'],
    ['e', 'e', 'e', 'x2', 'e', 'e', '+', 'x', 'e', 'e', 'x2', 'e', 'e', 'e'],
    ['e', '/', 'e', 'e', 'x2', 'e', '+', 'x', 'e', 'x2', 'e', 'e', '/', 'e'],
    ['e', 'e', '-', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', '-', 'e', 'e'],
    ['x3', 'e', 'e', 'x', '+', 'e', '1', '2', 'e', 'x', '+', 'e', 'e', 'x3'],
    ['x3', 'e', 'e', 'x', '+', 'e', '3', '4', 'e', 'x', '+', 'e', 'e', 'x3'],
    ['e', 'e', '-', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', '-', 'e', 'e'],
    ['e', '/', 'e', 'e', 'x2', 'e', '+', 'x', 'e', 'x2', 'e', 'e', '/', 'e'],
    ['e', 'e', 'e', 'x2', 'e', 'e', '+', 'x', 'e', 'e', 'x2', 'e', 'e', 'e'],
    ['e', 'e', 'x2', 'e', 'e', '-', 'e', 'e', '-', 'e', 'e', 'x2', 'e', 'e'],
    ['e', 'x2', 'e', 'e', '/', 'e', 'e', 'e', 'e', '/', 'e', 'e', 'x2', 'e'],
    ['x3', 'e', 'e', 'e', 'e', 'e', 'x3', 'x3', 'e', 'e', 'e', 'e', 'e', 'x3']]

In [11]:
numbers = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
           '14', '15', '16', '17', '18', '19', '20', '21', '24', '25', '27', '28',
           '30', '32', '35', '36', '40', '42', '45', '48', '49', '50', '54', '56', 
           '56', '60', '63', '64', '70', '72', '80', '81', '90']

In [12]:
def get_game_board(img):
    full_board = get_full_board(img)
    small_perct = 0.1293
    big_perct = 0.8707
    top_left = (small_perct * width, small_perct * width)
    top_right = (big_perct * width, small_perct * width)
    bottom_right = (big_perct * width, big_perct * width)
    bottom_left = (small_perct * width, big_perct * width)
    
    puzzle = np.array((top_left, top_right, bottom_right, bottom_left), dtype='float32')
    destination = np.array(([0, 0], [width, 0], [width, height], [0, height]), dtype='float32')
    M = cv.getPerspectiveTransform(puzzle, destination)
    
    board_copy = full_board.copy()
    board_copy = cv.warpPerspective(full_board, M, (width, height), flags=cv.INTER_LINEAR)
    # show_image("result", board_copy)
    return board_copy

In [13]:
def get_boards(folder):
    boards = []
    files = os.listdir(folder)
    for file in files:
        if file.endswith('.jpg'):
            img = cv.imread(folder + '/' + file)
            board = get_game_board(img)
            boards.append(board)
    return boards

In [14]:
def preprocess_image(img):
    img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    img_m_blur = cv.medianBlur(img_gray, 5)
    img_g_blur = cv.GaussianBlur(img_m_blur, (0, 0), 5)
    img_sharpened = cv.addWeighted(img_m_blur, 1.2, img_g_blur, -0.8, 0)
    _, thresh = cv.threshold(img_sharpened, 30, 255, cv.THRESH_BINARY)
    kernel = np.ones((5, 5), np.uint8)
    thresh = cv.erode(thresh, kernel)
    return img_gray

In [12]:
def create_templates():
    os.makedirs("templates", exist_ok=True)
    img = get_game_board(cv.imread("imagini_auxiliare/03.jpg"))
    processed_img = preprocess_image(img)
    
    index = 0
    for i in range(5, 10):
        for j in range(4, 12):
            x = j * square_size
            y = i * square_size
            template = processed_img[y+10:y+square_size-10, x+10:x+square_size-10]
            cv.imwrite(f"templates/{index}.jpg", template)
            index += 1
    for j in range(4, 10):
        x = j * square_size
        y = 10 * square_size
        template = processed_img[y+10:y+square_size-10, x+10:x+square_size-10]
        cv.imwrite(f"templates/{index}.jpg", template)

In [13]:
def create_templates_2():
    os.makedirs("templates", exist_ok=True)
    img = get_game_board(cv.imread("imagini_auxiliare/04.jpg"))
    processed_img = preprocess_image(img)
    
    index = 0
    for i in range(0, 11, 2):
        for j in range(0, 13, 2):
            x = j * square_size
            y = i * square_size
            template = processed_img[y+10:y+square_size-10, x+10:x+square_size-10].copy()
            cv.imwrite(f"templates/{index}.jpg", template)
            index += 1
            
    for j in range(0, 7, 2):
        x = j * square_size
        y = 12 * square_size
        template = processed_img[y+10:y+square_size-10, x+10:x+square_size-10].copy()
        cv.imwrite(f"templates/{index}.jpg", template)
        index += 1

In [14]:
create_templates_2()

In [15]:
def resize_templates():
    os.makedirs("templates_3", exist_ok=True)
    for i in range(46):
        img = cv.imread(f"templates_2/{i}.jpg")
        img = cv.resize(img, (50, 50))
        cv.imwrite(f"templates_3/{i}.jpg", img)

In [16]:
resize_templates()

In [ ]:
board = get_game_board(cv.imread("antrenare/1_01.jpg"))
show_image("board", board)
print(board.shape)

In [ ]:
empty_board_img = get_game_board(cv.imread("imagini_auxiliare/01.jpg"))

In [17]:
test_boards = get_boards("evaluare/fake_test")

In [18]:
def get_valid_posisions(board):
	valid_positions = []
	for i in range(14):
		for j in range(14):
			if board[i][j] in numbers:
				continue
			if i - 1 >= 0 and board[i-1][j] in numbers:
				valid_positions.append((i, j))
			elif i + 1 < 14 and board[i+1][j] in numbers:
				valid_positions.append((i, j))
			elif j - 1 >= 0 and board[i][j-1] in numbers:
				valid_positions.append((i, j))
			elif j + 1 < 14 and board[i][j+1] in numbers:
				valid_positions.append((i, j))
	return valid_positions

In [1]:
def classify_number(patch):
    patch = preprocess_image(patch)
    # show_image("patch", patch)
    max_corr = -np.inf
    index = -1
    for j in range(0, 46):
        img_template = cv.imread('templates_3/' + str(j) + '.jpg')
        if len(img_template.shape) == 3:
            img_template = cv.cvtColor(img_template, cv.COLOR_BGR2GRAY)
        img_template = cv.resize(img_template, (60, 60))
        show_image("template", img_template)
        show_image("patch", patch)
        corr = cv.matchTemplate(patch, img_template, cv.TM_CCOEFF_NORMED)
        corr = np.max(corr)
        if corr > max_corr:
            max_corr = corr
            index = j
    return numbers[index]

In [2]:
def solve(boards):
    os.makedirs("evaluare/fisiere_solutie/332_Hodivoianu_Anamaria", exist_ok=True)
    num_boards = len(boards)
    num_games = num_boards // 50
    for i in range(num_games):
        game_board = deepcopy(empty_board)
        curr_boards = boards[i*50 : (i+1)*50]
        curr_boards.insert(0, empty_board_img)
        for j in range(1, 51):
            diff = cv.subtract(curr_boards[j], curr_boards[j-1])
            valid_positions = get_valid_posisions(game_board)
            max_diff, max_x, max_y = 0, 0, 0
            for x in range(0, 14):
                for y in range(0, 14):
                    if (x, y) not in valid_positions:
                        continue
                    x_coord = x * square_size
                    y_coord = y * square_size
                    square = diff[x_coord:x_coord+square_size, y_coord:y_coord+square_size]
                    if np.mean(square) > max_diff:
                        max_diff = np.mean(square)
                        max_x = x
                        max_y = y
            # cv.rectangle(diff, (max_y*square_size, max_x*square_size), (max_y*square_size+square_size, max_x*square_size+square_size), (0, 0, 255), 10)
            # show_image("diff", diff)
            patch = curr_boards[j][max_x*square_size:max_x*square_size+square_size, max_y*square_size:max_y*square_size+square_size].copy()
            number = classify_number(patch)
            game_board[max_x][max_y] = number
            if j < 10:
                  with open("evaluare/fisiere_solutie/332_Hodivoianu_Anamaria/" + str(i + 1) + "_0" + str(j) + ".txt", "w") as f:
                        f.write(str(max_x + 1) + chr((max_y)%26+ord("A")) + " " + game_board[max_x][max_y])
            else:
                with open("evaluare/fisiere_solutie/332_Hodivoianu_Anamaria/" + str(i + 1) + "_" + str(j) + ".txt", "w") as f:
                    f.write(str(max_x + 1) + chr((max_y)%26+ord("A")) + " " + game_board[max_x][max_y])

In [3]:
solve(test_boards)

NameError: name 'test_boards' is not defined

In [ ]:
# find_color_values_using_trackbar(result)

In [ ]:
# # l = np.array([0, 0, 0])
# # u = np.array([179, 123, 255])

# # l = np.array([57, 23, 83])
# # u = np.array([179, 255, 255])

# # pt patratele portocalii
# # l = np.array([89, 0, 87])
# # u = np.array([179, 255, 255])

# # l = np.array([84, 0, 0])
# # u = np.array([179, 255, 255])

# # pt patrate
# l = np.array([93, 101, 80])
# u = np.array([111, 255, 255])

# result_hsv = cv.cvtColor(result, cv.COLOR_BGR2HSV)
# mask_hsv = cv.inRange(result_hsv, l, u)

# show_image('img_initial', result)
# show_image('mask_hsv', mask_hsv)

In [ ]:
# contours, _ = cv.findContours(mask_hsv, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

# result_copy = result.copy()
# cv.drawContours(result_copy, contours, -1, (0, 255, 0), 10)
# show_image('result', result_copy)

In [ ]:
# result_copy = result.copy()
# for c in contours:
# 	area = cv.contourArea(c)
# 	x, y, w, h = cv.boundingRect(c)
# 	cv.rectangle(result_copy, (x, y), (x+w, y+h), (0, 255, 0), 10)
# 	# x1, y1 = c[0][0]
# 	# cv.drawContours(result_copy, [c], -1, (0, 255, 0), 10)
# 	# x, y, w, h = cv.boundingRect(c)
# 	# if w > 1000 and h > 1000:
# 	# 	print(x, y, w, h)
# 	# 	cv.rectangle(result_copy, (x, y), (x+w, y+h), (0, 255, 0), 10)

# show_image("Shapes", result_copy)

In [ ]:
# image = cv.cvtColor(mask_hsv, cv.COLOR_2GRAY)
# image_m_blur = cv.medianBlur(image, 5)
# image_g_blur = cv.GaussianBlur(image_m_blur, (0, 0), 5) 
# image_sharpened = cv.addWeighted(image_m_blur, 1.5, image_g_blur, -1.3, 0)
# show_image('image_sharpened', image_sharpened)
# _, thresh = cv.threshold(image_sharpened, 30, 255, cv.THRESH_BINARY)

# kernel = np.ones((5, 5), np.uint8)
# thresh = cv.erode(thresh, kernel)
# show_image('image_thresholded', thresh)

# edges = cv.Canny(thresh, 200, 400)
# show_image('edges', edges)
# contours, _ = cv.findContours(edges, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
# max_area = 0

# result_copy = result.copy()
# cv.drawContours(result_copy, contours, -1, (0, 255, 0), 10)
# show_image('result', result_copy)

error: OpenCV(4.10.0) d:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.simd_helpers.hpp:92: error: (-2:Unspecified error) in function '__cdecl cv::impl::`anonymous-namespace'::CvtHelper<struct cv::impl::`anonymous namespace'::Set<3,-1,-1>,struct cv::impl::A0x9faa3a91::Set<3,4,-1>,struct cv::impl::A0x9faa3a91::Set<0,5,-1>,4>::CvtHelper(const class cv::_InputArray &,const class cv::_OutputArray &,int)'
> Invalid number of channels in input image:
>     'VScn::contains(scn)'
> where
>     'scn' is 1
